In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import colors
from netCDF4 import Dataset
import pandas as pd
import properscoring as ps
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import warnings
import xarray as xr
from matplotlib import cm
from utils.evaluation import plot_accumulated,find_landfalling_tcs,tc_region,create_xarray,get_storm_coords
from utils.metrics import calculate_crps
from global_land_mask import globe
from scipy.interpolate import griddata
# from utils.metrics import calculate_fid
import xesmf as xe
warnings.filterwarnings("ignore")

sns.set_style("white")
sns.set_palette(sns.color_palette("Paired"))

In [8]:
mode = 'validation'
mode = 'extreme_valid'
real = np.load('/user/home/al18709/work/dsrnngan_predictions/%s_real-opt.npy' % mode)[0][:,:,:,0]
pred = np.load('/user/home/al18709/work/dsrnngan_predictions/%s_pred-opt.npy' % mode)[0][:,:,:,0]
inputs = np.load('/user/home/al18709/work/dsrnngan_predictions/%s_input-opt.npy' % mode)[0][:,:,:,0]
meta = pd.read_csv('/user/work/al18709/tc_data_mswep/extreme_valid_meta.csv')
tcs = pd.read_csv('/user/work/al18709/ibtracks/tc_files.csv')

In [9]:
print(real.shape)
print(meta.shape)

(2000, 100, 100)
(5425, 5)


In [10]:
# get list of landfalling tcs
landfall_sids = find_landfalling_tcs(meta)
print(landfall_sids)
# assign sid variable to list of sid indices correspoinnding to storm timesteps
for sid in landfall_sids:
	indices = meta.sid[meta.sid == sid].index.tolist()
	exec('sid_%s = indices' % sid)

all_sids = list(dict.fromkeys(meta['sid']))
print(len(all_sids)) # not sure all sids are being found in the metadata?

['1999151N09132', '2008252N16128', '2004061S12072', '1988308N09140', '2012296N06135', '1998232N15312', '1999260N20130', '2011192N18158', '2004239N11171', '1991338S08181', '1993253N06150', '2015211N13162', '2014068S16169', '2005192N22155', '2010070S15168', '2007274N18131', '2005268N19146', '1995348S15135', '1993224N07153', '1993258N11279', '2019243N06136', '2001309N10130', '1999318N17278', '1988253N12306', '1988307N09131', '1991226N18159', '1982202N11165', '1996079S12170', '1993252N11265', '2007297N18300', '1988055S10180', '1979275N06159', '1993240N17142', '1992289N08135', '2004219N15137', '1993360S04171', '2003061S21148', '1981158N17119', '2009164N10131', '1988295N10138', '1997062S16158', '1998259N10335', '1988006S05182', '2003359S15177', '2016256N12146', '2003011S09182', '2014275N06166', '1996260N19107', '2000230N08139', '1996289N13280', '1995271N19273', '1999253N17124', '2009268N14128']
100


In [5]:
# grab mswep coordinate variables
fp = '/bp1store/geog-tropical/data/Obs/MSWEP/3hourly_invertlat/2000342.00.nc'
d = Dataset(fp, 'r')
lat = d.variables['lat'][:] #lat
lon = d.variables['lon'][:] #lon

In [15]:
# clip to entire TC region
# lats,lons = tc_region(meta,sid_2008169N08135,lat,lon)
lats,lons = tc_region(meta,sid_1999151N09132,lat,lon)

In [16]:

# initialise accumulated xarray
grid_x, grid_y = np.meshgrid(lats, lons)
a = np.zeros((grid_x.shape))
accumulated_ds = create_xarray(lats,lons,a)
accumulated_ds_pred = create_xarray(lats,lons,a)
accumulated_ds_input = create_xarray(lats,lons,a)


# accumulated = np.zeros(grid_x.shape)
for i in sid_1999151N09132:
	storm_lats,storm_lons = get_storm_coords(lat,lon,meta,i)
	ds = create_xarray(storm_lats,storm_lons,real[i])
	ds_pred = create_xarray(storm_lats,storm_lons,pred[i])
	input_lats,input_lons = get_storm_coords(np.arange(-89.5,90,1),np.arange(-179.5,180),meta,i)
	ds_input = create_xarray(input_lats,input_lons,inputs[i])

	# regrid so grids match
	regridder = xe.Regridder(ds, accumulated_ds, "nearest_s2d")
	ds_out = regridder(ds)
	ds_pred_out = regridder(ds_pred)

	# regird the inputs
	# regridder = xe.Regridder(ds_input, accumulated_ds, "bilinear")
	regridder = xe.Regridder(ds_input, accumulated_ds, "nearest_s2d")
	ds_input_out = regridder(ds_input)

	# add up rainfall
	accumulated_ds = accumulated_ds + ds_out
	accumulated_ds_pred = accumulated_ds_pred + ds_pred_out
	accumulated_ds_input = accumulated_ds_input + ds_input_out

# hr_ds = create_xarray(storm_lats,storm_lons,real[i])

# plot accumulated rainfall
# https://earthobservatory.nasa.gov/images/8868/rainfall-from-typhoon-fengshen
plot_accumulated(accumulated_ds['precipitation'],accumulated_ds['lat'].values,accumulated_ds['lon'].values,vmin=0,vmax=600,levels = [0,50,100,150,200,250,300,350,400,450,500,550,600,650,700],plot='show')
plot_accumulated(accumulated_ds_pred['precipitation'],accumulated_ds_pred['lat'].values,accumulated_ds_pred['lon'].values,vmin=0,vmax=600,levels = [0,50,100,150,200,250,300,350,400,450,500,550,600,650,700],plot='show')
plot_accumulated(accumulated_ds_input['precipitation'],accumulated_ds_input['lat'].values,accumulated_ds_input['lon'].values,vmin=0,vmax=600,levels = [0,50,100,150,200,250,300,350,400,450,500,550,600,650,700],plot='show')

# real_accumulated = accumulated_ds

anomaly = accumulated_ds_pred - accumulated_ds
rmse_hr = np.sqrt(((accumulated_ds_pred - accumulated_ds) ** 2).mean()).values()
print(anomaly.max())
plot_accumulated(anomaly['precipitation'],anomaly['lat'].values,anomaly['lon'].values,plot='show',vmin=-40,vmax=40,levels = [0,10,20,30,40],cmap='coolwarm')

anomaly_lr = accumulated_ds_input - accumulated_ds
plot_accumulated(anomaly_lr['precipitation'],anomaly_lr['lat'].values,anomaly_lr['lon'].values,plot='show',vmin=-40,vmax=40,levels=[0,10,20,30,40],cmap='coolwarm')
rmse_lr = np.sqrt(((accumulated_ds_input - accumulated_ds) ** 2).mean()).values()
print(rmse_hr)
print(rmse_lr)
print(anomaly_lr.max())


In [ ]:
train_X = np.load('/user/work/al18709/tc_data_mswep_cat1plus/train_X.npy')
print(train_X.shape)
test_X = np.load('/user/work/al18709/tc_data_mswep_cat1plus/test_X.npy')
print(test_X.shape)
valid_X = np.load('/user/work/al18709/tc_data_mswep_cat1plus/valid_X.npy')
print(valid_X.shape)
extreme_valid = np.load('/user/work/al18709/tc_data_mswep_cat1plus/extreme_valid_X.npy')
print(extreme_valid.shape)
extreme_test = np.load('/user/work/al18709/tc_data_mswep_cat1plus/extreme_test_X.npy')
print(extreme_test.shape)